In [1]:
import os, re, sys
import numpy as np
import pandas as pd

import rowan                       # conda install -c conda-forge rowan
import ipyvolume as ipv            # conda install -c conda-forge ipyvolume
from phonlab.utils import dir2df   # pip install git+https://github.com/rsprouse/phonlab

# Load inventory of files

In [2]:
datadir = 'data/'

# Subject we will analyze.
subj = 'knight'

# Filenaming like:
# knight_occlusal_001.wav, knight_no_ref_000.tsv, knight_palate_002.wco
acqpat = re.compile(
    '''
    ^                      # filename starts with...
    (?P<sname>[^_]+)       # subject name (series of non-underscore chars)
    _                      # underscore
    (?P<acqtype>.+)        # acquisition type (series of chars, including underscore)
    _                      # underscore
    (?P<trial>\d+)         # left-padded integer trial number
    \.(tsv|wav|wco)        # file extension
    $                      # ...and no more
    ''',
    re.VERBOSE
)

# These aren't used in this notebook.
sensors = ["REF","UL","LL","JW","TT","TB","TD","TL","LC","UI","J","OS","MS","PL"]
subcolumns = ["ID","frame","state","q0","qx","qy","qz","x","y","z"] 

In [3]:
detdf = pd.read_excel(os.path.join(datadir, 'subject_details.xlsx'))
detdf

,subject,occlusal,palate,rightMastoid,leftMastoid,nasion,tongueBack,tongueDorsum,tongueBlade,tongueBody,...,frontOcclusal,palateTrace,palateStart,palateEnd,ceilingWindowFlag,ceilingWindowLength,lowPassFlag,lowPassHz,percentBackCutoff,percentFrontCutoff
0,subject_K01,K01/knight_occlusal_000,K01/knight_occlusal_001,10,9,11,-1,-1,-1,-1,...,7,-1,0,10000,1,0.1,1,0.125,8,8


Map sensor names to abbreviations. (The mapping itself is currently unused, but the list of sensor names `sensname` is used.)

In [4]:
name2abbr = {
    'rightMastoid': 'rMS',
    'leftMastoid': 'lMS',
    'nasion': 'NS',
    'tongueBack': 'TB',
    'tongueDorsum': 'TD',
    'tongueBlade': 'TBl',
    'tongueBody': 'TBd',
    'tongueLeft': 'lTNG',
    'tongueTip': 'TT',
    'tongueRight': 'rTNG',
    'upperIncisor': 'UI',
    'lowerIncisor': 'LI',
    'upperLip': 'UL',
    'lowerLip': 'LL',
    'rightUltrasound': 'rUS',
    'leftUltrasound': 'lUS',
    'frontUltrasound': 'fUS',
    'rightOcclusal': 'rOC',
    'leftOcclusal': 'lOC',
    'frontOcclusal': 'fOC',
}
sensnames = list(name2abbr.keys())

# Doublecheck for duplicate abbreviations.
try:
    assert(sorted(list(set(name2abbr.values())))) == sorted(list(name2abbr.values()))
except AssertionError:
    sys.stderr.write('WARNING: name2abbr has a duplicated abbreviation.\n')

Set up the sensor map, based on the subject_details file. This is specific to the K01 dataset. For other datasets, construct a dict that maps sensor keys to channel index.

In [5]:
sampsubj = detdf[detdf.subject == 'subject_K01'].iloc[0]
# Subtract 1 from sensor number since this is Python, not Matlab.
smap = {
    n: sampsubj[n]-1 for n in sampsubj.index if n in sensnames and sampsubj[n] > 0
}
smap

{'rightMastoid': 9,
 'leftMastoid': 8,
 'nasion': 10,
 'upperIncisor': 3,
 'lowerIncisor': 5,
 'lowerLip': 4,
 'rightOcclusal': 7,
 'leftOcclusal': 0,
 'frontOcclusal': 6}

In [6]:
sfiles = dir2df(
    datadir,
    fnpat=acqpat,
    addcols=['dirname', 'barename', 'ext', 'bytes']
)
sfiles

,dirname,relpath,fname,barename,ext,bytes,sname,acqtype,trial
0,data/,.,knight_no_ref_000.tsv,knight_no_ref_000,.tsv,10184962,knight,no_ref,000
1,data/,.,knight_no_ref_000.wav,knight_no_ref_000,.wav,2654208,knight,no_ref,000
2,data/,.,knight_no_ref_000.wco,knight_no_ref_000,.wco,3138,knight,no_ref,000
3,data/,.,knight_occlusal_001.tsv,knight_occlusal_001,.tsv,659738,knight,occlusal,001
4,data/,.,knight_occlusal_001.wav,knight_occlusal_001,.wav,180224,knight,occlusal,001
5,data/,.,knight_occlusal_001.wco,knight_occlusal_001,.wco,3138,knight,occlusal,001
6,data/,.,knight_with_ref_001.tsv,knight_with_ref_001,.tsv,9538730,knight,with_ref,001
7,data/,.,knight_with_ref_001.wav,knight_with_ref_001,.wav,2646016,knight,with_ref,001
8,data/,.,knight_with_ref_001.wco,knight_with_ref_001,.wco,2956,knight,with_ref,001


Do some data checking. The next cell checks to see whether each acquisition has the full set of .tsv, .wav, and .wco files and will print acquisitions that lack any of those. If all acquisitions are complete the output will be empty.

In [7]:
ftypes = set(['.tsv', '.wav', '.wco'])
sfiles.groupby(['acqtype', 'trial']).apply(
    lambda x: x if ftypes.difference(x.ext.values) != set() else None
)

""


Doublecheck there are no empty files. The expected output is an empty dataframe.

In [8]:
sfiles[sfiles.bytes == 0]

,dirname,relpath,fname,barename,ext,bytes,sname,acqtype,trial


# Define some functions

These should be moved to a separate file when ready.

In [9]:
class NDIData(object):
    '''
    A representation of NDI Wave tsv data.
    
    Instantiate the NDIData object with the path to a .tsv file and mapping of
    sensor names to channel index in the .tsv file. The .tsv file is expected to
    have one (time) column at the left, followed by repetitions of sets
    of columns consisting of a 'State' column (and possibly others, e.g.
    channel name and frame identifier) followed by columns Q0, Qx, Qy, Qz, Tx, Ty, Tz
    in exactly that order. These repetitions are expected to occur regularly
    with no exception, other than the possibility of empty columns identified
    in the header with ' ', which are ignored. The Q0...Tz columns may have
    suffixes appended to them, either in the .tsv file or as a side effect of
    `read_csv`.
    
    The sensor map is a dictionary of the form {'sensor_name': idx, ...}, with the
    index 0 indicating the first repetition of Q0...Tz columns and index 1 for the
    second repetition.
    
    By default the 'State' values are processed to replace Q0...Tz values with
    `Nan` for any sensor-frame that is not 'OK'. Replacement occurs automatically
    when the `NDIData` object is instantiated unless `replace_bad` is set to
    `False`.
    
    Access to the .tsv data is available as a dataframe via the `df` attribute:
    
    ```
    tsv = NDIData(tsvpath, sensormap)
    tsv.df
    ```
    
    Convenience methods are provided to access the Q0...Tz columns for all sensors or a
    set of sensors as a 3d numpy ndarray, with dimensions 0) frame index (time);
    1) sensors; 2) Q0...Tz. Use `qtvals` to return columns [`Q0`, `Qx`, `Qy`, `Qz`,
    `Tx`, `Ty`, `Tz`] for a given set of sensors. Use `qvals` to return only
    [`Q0`, `Qx`, `Qy`, `Qz`], and use `tvals` to return only [`Tx`, `Ty`, `Tz`] for
    the given sensors.
    
    ```
    # Return 3d ndarray (axes: time, sensor, Q0...Tz)
    tsv.qtvals()                          # Return Q0, Qx, Qy, Qz, Tx, Ty, Tz for all sensors
    tsv.qvals(['nasion', 'leftMastoid'])  # Return Q0, Qx, Qy, Qz for two sensors
    tsv.tvals(['nasion', 'leftMastoid'])  # Return Tx, Ty, Tz for two sensors
    '''

    def __init__(self, tsvname, sensormap, replace_bad=True):
        ''''''
        self.df = pd.read_csv(
            tsvname,
            sep='\t',
            # Skip empty columns, which have a whitespace-only header.
            usecols=lambda head: head != ' ',
        )
        self.sensormap = sensormap
        self.rev_sensormap = {v: k for k, v in sensormap.items()}
        if replace_bad is True:
            self._set_bad_to_nan()
        # TODO: relabel columns with sensor names?

    def _set_bad_to_nan(self):
        '''Set values of Q/T columns to NaN if corresponding 'State' column
        is not 'OK'.'''
        bad = self.df.loc[:,self.df.columns.str.match('[Ss]tate')] != 'OK'
        qtvals = self.qtvals()
        qtvals[bad] = np.nan  # bad should broadcast along Q0Txyz dimension        
        self.replace_columns(qtvals, qt='QT') # Put new qtvals back into dataframe.
    
    def qtindexes(self, qt, sensors=None):
        '''Return a list of column integer indexes in self.df for the Q/T columns for
        given sensors. The Q/T columns for a given sensor are assumed to be
        in the order Q0, Qx, Qy, Qz, Tx, Ty, Tz with no other columns
        intervening.
            
        This should be robust whether the Q/T values have additional
        suffixes or not, e.g. Q0, Q0.1, Q0.15, etc.
            
        The value of sensors should be a list of keys in self.sensormap.
        If sensors is None, include all sensors.
        '''

        # Get indexes of all Q0/Tx columns.
        qtmap = {'Q': 'Q0', 'T': 'Tx', 'QT': 'Q0'}
        q0s = (
            self.df.columns.str.match('^{:}'.format(qtmap[qt]))
        ).nonzero()[0]

        # Test our assumption that Q/T0 channels are equally-spaced.
        try:
            assert(np.diff(q0s).std() == 0)
        except AssertionError:
            msg = 'Q/T channels are not equally-spaced!\n'
            raise RuntimeError(msg)

        # Calculate the indexes of the selected Q0/Tx columns.
        step = int(np.diff(q0s)[0])
        # First get sensor indexes, e.g. in range 0-15.
        if sensors is None:  # Use all sensors.
            snums = np.arange(len(q0s))
        else:
            snums = np.array([self.sensormap[n] for n in sensors])
        # Multiply by step and add column offset from left.
        snums = (snums * step) + q0s[0]
        # Add the x,y,z column indexes and flatten the list.
        if qt == 'Q':
            qnums = [(n, n+1, n+2, n+3) for n in snums]
        elif qt == 'T':
            qnums = [(n, n+1, n+2) for n in snums]
        elif qt == 'QT':
            qnums = [(n, n+1, n+2, n+3, n+4, n+5, n+6) for n in snums]
        flatnums = [item for sublist in qnums for item in sublist]
        return flatnums

    def replace_columns(self, vals, qt, sensors=None):
        '''Replace column data in self.df with new values from an ndarray.
        
        Parameters
        ----------
        
        vals: 2D or 3D ndarray
            New values for replacement. If 3D, `vals` has axes (time, sensors, Q0...Tz)
            and will be reshaped to 2D to match the dataframe column arrangement; the
            latter two dimensions are collapsed. An input 2D array must have the same
            axes arrangement as the reshaped 3D array.
            
        qt: str ('QT', 'Q', or 'T')
            The kinds of new column data in `vals`. Use 'QT' if replacing all 'Q' and
            'T' values. Use 'Q' if replacing only 'Q' values and 'T' if only 'T' values.
            If `vals` is 3D, then the shape of second dimension must match the `qt`
            value: {'QT': 7, 'Q': 4, 'T': 3}.
            
        sensors: list of str
            List of sensor values in `vals`. For 3D `vals` the third axis must be the
            same length as `sensors`.

        Returns
        -------
        Returns True on success. Raises an error if assignment does not succeed.

        '''
        try:
            assert(len(vals) == len(self.df))
        except AssertionError:
            msg = "Can't replace columns. New values are not equal in length to old.\n"
            raise ValueError(msg)
        if len(vals.shape) == 3:
            vals = vals.reshape(len(vals), -1)
        try:
            self.df.iloc[:, self.qtindexes(qt=qt, sensors=sensors)] = vals
        except:
            msg = "Could not replace columns with new data.\n"
            raise ValueError(msg)
        return True

    def qtvals(self, sensors=None):
        '''Return the Q0, Qx, Qy, Qz, Tx, Ty, Tz values for given sensors
        as a 3d ndarray. If sensors is None, return all sensors.
        
        The dimensions are:
            frame index (time)
            sensors
            0xyz,xyz coordinates
        '''
        d = self.df.iloc[:, self.qtindexes('QT', sensors)]
        return d.values.reshape(len(self.df), -1, 7)

    def qvals(self, sensors=None):
        '''Return the Q0, Qx, Qy, Qz values for given sensors as a 3d ndarray.
        If sensors is None, return all sensors.
        
        The dimensions are:
            frame index (time)
            sensors
            0xyz values
        '''
        d = self.df.iloc[:, self.qtindexes('Q', sensors)]
        return d.values.reshape(len(self.df), -1, 4)

    def tvals(self, sensors=None):
        '''Return the Tx, Ty, Tz values for given sensors as a 3d ndarray.
        If sensors is None, return all sensors.
        
        The dimensions are:
            frame index (time)
            sensors
            xyz coordinates
        '''
        d = self.df.iloc[:, self.qtindexes('T', sensors)]
        return d.values.reshape(len(self.df), -1, 3)

def ideal_biteplate(pts, right=0, left=1, front=2):
    '''Calculate angles and length of the triangle formed by the
occlusal biteplate sensors. Return the coordinates of an identical
triangle that has the desired location and orientation.
# TODO: filter out frames if status != 'OK'
pts : 3D or 2D ndarray
    Axes are:
        triangle index, (normally time or frame number)
        sensors (default [occlusalRight, occlusalLeft, occlusalFront])
        coordinates [x,y,z],
    If a 2D array is passed, the value is assumed to be a single triangle
    with sensors comprising the first dimension and coordinates [x,y,z]
    the second.

right : int (default 0)
    The index in the third dimension of points where the occlusalRight
    sensor is located.

left : int (default 1)
    The index in the third dimension of points where the occlusalLeft
    sensor is located.

front : int (default 2)
    The index in the third dimension of points where the occlusalFront
    sensor is located.

Returns
-------
ideals : 3D or 2D ndarray
    Return an array of idealized triangle point coordinates that define triangles
    of the same size and shape as the input triangles and situated into a 
    desired reference orientation. If the input is 2D (a single triangle), then
    the returned value is also 2D.

    In the reference orientation each triangle lies on the xy plane where z=0.
    The occlusalRight and occlusalLeft sensors are placed on the x axis (x=0) and
    equidistant from the origin, i.e. they are centered at y=0.
    
    The occlusalFront sensor is placed in the x dimension at the height of the
    triangle from the base defined by the right and left sensors. If the input
    triangles were perfect isosceles the occlusalFront sensor would be on the
    x axis (y=0), but we assume that they are not perfect and calculate the
    y dimension for the occlusalFront sensor as well.
# TODO: return mean idealbp
'''
    # OR = occlusalRight (right)
    # OL = occlusalLeft (left)
    # OF = occlusalFront (front)
#           OF
#          /|\
#         b h c
#        /  |  \
#       OR--a--OL
#       Angle at OL = alpha

    return_2d = False
    if len(pts.shape) == 2:
        pts = np.array([pts])
        return_2d = True
    try:
        assert(len(pts.shape) == 3 and pts.shape[1:3] == (3, 3))
    except AssertionError:
        msg = 'Input must be shape (N, 3, 3) or (3, 3)\n.'
        raise RuntimeError(msg)

    # Map names of axes to their indexes.
    [x, y, z] = [0, 1, 2]  # indexes of coordinate labels in second dimension of pts

    # Initialize all point coordinates to (0,0,0).
    ideals = np.zeros_like(pts)

    # Find lengths of the triangle sides with the Distance Formula (3d):
    # √(
    #    (x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2
    #  )
    [a, b, c] = [
        np.sqrt(
            np.sum(
                np.square(
                    pts[:,p2,[x,y,z]] - pts[:,p1,[x,y,z]]
                ),
            axis=1)
        ) for p2, p1 in [(right, left), (right, front), (left, front)]
    ]

    # Step 1. occlusalRight and occlusalLeft
    # Put points OR and OL along y axis at x,z=0, with length a and centered
    # at y=0, i.e. put OL at y = (half of a) and OR at y = -(half of a).
# TODO: does this swap signs?
    ideals[:,left,y] = a / 2
    ideals[:,right,y] = -ideals[:,y,left]

    # Step 2. occlusalFront
    # Calculate the area of the triangle from the sides using Heron's formula.
    s = (a + b + c) / 2  # Semi-perimeter of the triangle.
    area = np.sqrt(s * (s - a) * (s - b) * (s - c)) # Heron's formula.
    
    # Calculate the height of the triangle from side a to OF using Area Formula.
    h = (2 * area) / a

    # Put point OF at distance h in x dimension.
    ideals[:,front,x] = h
    
    # Put point OF at correct distance in y dimension. If the biteplate were
    # perfectly constructed this would be at y=0.
    # Use the Law of Cosines to find angle alpha (OL), then calculate y
    # value of point OF.
    alpha = np.arccos(
        ((a**2) + (b**2) - (c**2)) / (2 * a * b)
    )
    ideals[:,front,y] = (a / 2) - (np.cos(alpha) * c)
    if return_2d is True:
        ideals = ideals.squeeze()
    return ideals

def transform_frame(hdvals, idealhd, allvals):
    '''Transform a frame of sensor data by comparing fixed head sensor values to
    ideal head position.
    
    Parameters
    ----------
    
    hdvals: 2D array (shape: N, 3)
        Fixed head position sensor readings for the frame.
        
    idealhd: 2D array (shape: N, 3)
        Idealized fixed head positions.
        
    allvals: 2D array (shape: N, 3)
        All sensor values to be rotated.
        
    For all arrays, the first axis is sensors (N), and the second `xyz`.
    
    The first dimension of `hdvals` and `idealhd` must have corresponding sensors (N) in
    identical order (normally there are three).
    
    The `allvals` array can include all sensors and the sensors need not be the same
    size or in the same order as `hdvals` and `idealhd` (normally N > 3).
    
    Returns
    -------
    
    rotated: 2D array
        The rotated values.
    '''
    R, t = rowan.mapping.kabsch(hdvals, idealhd)
    q = rowan.from_matrix(R)  # Convert to quaternion
    return rowan.rotate(q, allvals) + t

Get metadata for the biteplate recording.

In [10]:
tsv = sfiles[
    (sfiles.barename.str.match('{:}_occlusal_'.format(subj))) & (sfiles.ext == '.tsv')
].iloc[0]
tsv

dirname                       data/
relpath                           .
fname       knight_occlusal_001.tsv
barename        knight_occlusal_001
ext                            .tsv
bytes                        659738
sname                        knight
acqtype                    occlusal
trial                           001
Name: 3, dtype: object

## Biteplate rotation

Load the biteplate recording. 

In [11]:
occ = NDIData(
    tsvname=os.path.join(tsv.dirname, tsv.fname),
    sensormap=smap
)

Use the `tvals` method get the `Tx`, `Ty`, and `Tz` columns for a list of sensors.

In [12]:
scalefactor = 0.01   # Scale to make plotting easier
bpsensors = ['rightOcclusal', 'leftOcclusal', 'frontOcclusal']
bpvals = occ.tvals(bpsensors) * scalefactor

Get idealized biteplate sensor values in reference orientation.

In [13]:
idealbp = np.nanmean(ideal_biteplate(bpvals), axis=0)

Calculate the rotation and translation from the mean input biteplate to the ideal biteplate.

In [14]:
R, t = rowan.mapping.kabsch(
    np.nanmean(bpvals, axis=0),
    idealbp
)
q = rowan.from_matrix(R)  # Convert to quaternion

As a check, transform `bpvals` using the calculated rotation and translation.

In [15]:
rotbp = rowan.rotate(q, bpvals) + t

## Plot biteplate rotation

Select a frame and see how the biteplate looks. The original biteplate is plotted with red diamonds at the vertices and a yellow surface. The target ideal biteplate is plotted with blue vertices, and the calculated rotation of the original biteplate is an orange surface. If the rotation was done well the orange surface should fit neatly in the blue vertices.

Note that ipyvolume plots require `x, y, z` as the first three arguments. We transpose the frame data to put the `xyz` axis first and use the splat operator `*` to separate the columns into the three input arguments.

In [16]:
frame = 0  # Select a frame to view.
ipv.clear()

# Original biteplate values.
ipv.scatter(*bpvals[frame,:,:].T, size=2, color='red', marker='diamond')  # ‘arrow’, ‘box’, ‘diamond’, ‘sphere’, ‘point_2d’, ‘square_2d’, ‘triangle_2d’, ‘circle_2d’
ipv.plot_trisurf(*bpvals[frame,:,:].T, triangles=[0, 1, 2], color='yellow')

# Ideal biteplate values.
ipv.scatter(*idealbp.T, size=2, color='blue')
ipv.plot_trisurf(*rotbp[frame,:,:].T, triangles=[0, 1, 2], color='orange')
ipv.show()

## Ideal head position

If the biteplate rotations look good, then apply the same transform to the head position sensors from the biteplate recording.

In [17]:
hdsensors = ['rightMastoid', 'leftMastoid', 'nasion']
hdvals = occ.tvals(hdsensors) * scalefactor
idealhd = rowan.rotate(q, hdvals) + t
idealhd = np.nanmean(idealhd, axis=0)

As a check, transform `hdvals` using the calculated rotation and translation.

In [18]:
rothd = rowan.rotate(q, hdvals) + t

## Plot ideal head position

As with the biteplate, the original head position is displayed with red vertices and a yellow surface. The target head position is shown by the blue vertices, and the rotated head positions are shown by the orange surface.

In [19]:
frame = 0  # Select a frame to view.
ipv.clear()

# Original biteplate values.
ipv.scatter(*hdvals[frame,:,:].T, size=2, color='red', marker='diamond')  # ‘arrow’, ‘box’, ‘diamond’, ‘sphere’, ‘point_2d’, ‘square_2d’, ‘triangle_2d’, ‘circle_2d’
ipv.plot_trisurf(*hdvals[frame,:,:].T, triangles=[0, 1, 2], color='yellow')

# Ideal biteplate values.
ipv.scatter(*idealhd.T, size=2, color='blue')
ipv.plot_trisurf(*rothd[frame,:,:].T, triangles=[0, 1, 2], color='orange')
ipv.show()

## Rotate a full acquisition

Get metadata for the acquisition.

In [20]:
tsv = sfiles[
    (sfiles.barename.str.match('{:}_with_ref_'.format(subj))) & (sfiles.ext == '.tsv')
].iloc[0]
tsv

dirname                       data/
relpath                           .
fname       knight_with_ref_001.tsv
barename        knight_with_ref_001
ext                            .tsv
bytes                       9538730
sname                        knight
acqtype                    with_ref
trial                           001
Name: 6, dtype: object

Load the recording.

*NOTE* The only metadata in subject_details.xls is related to the biteplate recording. We'll assume the sensormap from that recording is the same and reuse it. Unfortunately this sensor map does not list many sensors of interest, like the tongue sensors.

In [21]:
wref = NDIData(
    tsvname=os.path.join(tsv.dirname, tsv.fname),
    sensormap=smap
)

Get the head positions.

In [22]:
hdvals = wref.tvals(hdsensors) * scalefactor
hdvals.shape

(6004, 3, 3)

Also get all sensor values.

*NOTE* This is limited to sensors for which we can guess a mapping based on the biteplate acquisition.

In [23]:
sensors = hdsensors + ['upperIncisor', 'lowerIncisor', 'lowerLip']
tvals = wref.tvals(sensors) * scalefactor
tvals.shape

(6004, 6, 3)

For each frame, find transform to move the fixed head sensors to the ideal position and apply to all sensors.

In [24]:
rothd = np.zeros_like(tvals) * np.nan
for idx in np.arange(len(tvals)):
    try:
        rothd[idx,:,:] = transform_frame(hdvals[idx,:,:], idealhd, tvals[idx,:,:])
    except np.linalg.LinAlgError as e:
        msg = "Could not rotate frame {:d}. ".format(idx)
        sys.stderr.write(msg)
        sys.stderr.write(str(e) + '\n')

Could not rotate frame 696. SVD did not converge
Could not rotate frame 697. SVD did not converge
Could not rotate frame 698. SVD did not converge
Could not rotate frame 699. SVD did not converge
Could not rotate frame 700. SVD did not converge
Could not rotate frame 701. SVD did not converge
Could not rotate frame 1002. SVD did not converge
Could not rotate frame 1003. SVD did not converge
Could not rotate frame 1004. SVD did not converge
Could not rotate frame 1005. SVD did not converge
Could not rotate frame 1006. SVD did not converge
Could not rotate frame 1007. SVD did not converge
Could not rotate frame 1378. SVD did not converge
Could not rotate frame 1379. SVD did not converge
Could not rotate frame 1380. SVD did not converge
Could not rotate frame 1381. SVD did not converge
Could not rotate frame 1382. SVD did not converge
Could not rotate frame 1383. SVD did not converge
Could not rotate frame 1384. SVD did not converge
Could not rotate frame 1385. SVD did not converge
Could 

In [25]:
frame = 0  # Select a frame to view.
ipv.clear()

# Original head position values.
ipv.scatter(*hdvals[frame,:,:].T, size=2, color='red', marker='diamond')  # ‘arrow’, ‘box’, ‘diamond’, ‘sphere’, ‘point_2d’, ‘square_2d’, ‘triangle_2d’, ‘circle_2d’
ipv.plot_trisurf(*hdvals[frame,:,:].T, triangles=[0, 1, 2], color='yellow')

# Ideal head position values.
ipv.scatter(*idealhd.T, size=2, color='blue')
ipv.plot_trisurf(*rothd[frame,:,:].T, triangles=[0, 1, 2], color='orange')

# Rotated sensors plotted in green.
ipv.scatter(*rothd[frame,:,:].T, size=2, color='green')
ipv.show()